In [1]:
from data_functions import *
from ML_continuous import *

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import statsmodels.formula.api as smf

import random

In [2]:
for_pres = read('texas_president.xlsx')
for_pres = for_pres.rename(index=str, columns={"count": "county"})
for_pres = for_pres.rename(index=str, columns={"Urbanization Status": "urban", "Border Status": "border"})

In [3]:
for_house = read('texas_counties.xlsx')
for_house = for_house.rename(index=str, columns={"rep_number": "district_num", "candidate_name": "dem_candidate", "dem_vote_share": "two_party_vtshare", "Urbanization Status": "urban", "Border Status": "border"})
for_house = for_house[for_house.county != 'ALL COUNTIES']
print(for_house.shape)

(308, 4)


In [4]:
counties = list(for_pres.county)

county_nums = {}
val = 1
for c in counties:
    county_nums[c] = val
    val += 1
    
for_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])
for_house['office'] = 'house'
for_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]

## Info data frames will contain all the demographic etc information for each of those entities: county, pres candidate, or house candidate. To be combined after research is finished

In [5]:
county_info = for_pres[['county', 'county_num', 'urban', 'border']]
county_info['urban'] = county_info.urban.apply(lambda x: 1 if x == 'Urban' else 0)
county_info['border'] = county_info.urban.apply(lambda x: 1 if x == 'Border' else 0)

In [6]:
pres_info = for_pres[['office', 'dem_candidate', 'two_party_vtshare', 'candidate_male', 'candidate_white', 'candidate_hispanic']]
#pres_info = pres_candidates.groupby(['dem_candidate','office']).median()
#pres_info = pres_candidates.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [8]:
house_info = for_house[['district_num', 'office', 'dem_candidate', 'two_party_vtshare']]
house_info = house_info.groupby(['dem_candidate','office']).median()
house_info = house_info.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [10]:
house_info.to_csv('house_candidates_to_add_info.csv')

In [11]:
for_house = for_house.merge(county_info, on=['county', 'county_num'])
for_house.columns

Index(['county', 'district_num', 'dem_candidate', 'two_party_vtshare',
       'county_num', 'office', 'urban', 'border'],
      dtype='object')